# 03_experiments_tfidf_logistic_regression_bias_and_ablation.ipynb
# Experimentos de Viés e Análise de Componentes: Baseline (TF-IDF + Regressão Logística)

# Objetivo:
Experimentos para investigar viés/leakage no dataset:
- Remoção do termo "Reuters" do texto (sem remover registros).
- Inclusão da variável 'subject' como possível proxy editorial.

# O que ele faz:
1) EXPERIMENTO A (BASE): title+text
2) EXPERIMENTO B (REMOÇÃO): remover 'Reuters'
3) EXPERIMENTO C (METADADO): incluir 'subject'
4) EXPERIMENTO D: subject + sem Reuters
5) Tabela final dos resultados

# Observação:

É utilizado o modelo baseline (TF-IDF + Regressão Logística), mantendo a mesma configuração de hiperparâmetros empregada na etapa anterior (notebook 02).


In [1]:
# 0) Importação das bibliotecas e carregamento dos dados

import pandas as pd
import numpy as np
import re

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, classification_report

train = pd.read_csv("train.csv")
test  = pd.read_csv("test.csv")

In [2]:
# Pré-processamento dos dados
# Conteúdo base (igual ao baseline)

# Título e texto Concatenados para fornecer mais informação ao modelo
train["content"] = train["title"].fillna("") + " " + train["text"].fillna("")
test["content"]  = test["title"].fillna("")  + " " + test["text"].fillna("")

# remoção de duplicatas completas
train = train.drop_duplicates(subset=["title", "text"]).reset_index(drop=True)

# Definição das variáveis
X = train["content"]
y = train["label"]

In [3]:
# FUNÇÕES AUXILIARES


# Remove ocorrências comuns de Reuters (proxy editorial) e mantém o resto do texto intacto.
def remove_reuters(text: str) -> str:

    text = str(text)
    # remove "(Reuters)" e "Reuters" (case-insensitive)
    text = re.sub(r"\(reuters\)", " ", text, flags=re.IGNORECASE)
    text = re.sub(r"\breuters\b", " ", text, flags=re.IGNORECASE)
    # limpa espaços
    text = re.sub(r"\s+", " ", text).strip()
    return text


# Cria o pipeline TF-IDF + Logistic Regression, no estilo do baseline.
def make_pipeline(max_features=70000, c_value=3.0, ngram_range=(1,3)):

    pipeline = Pipeline([
        ("tfidf", TfidfVectorizer(
            stop_words="english",
            ngram_range=ngram_range,
            max_features=max_features,
            min_df=2,
            sublinear_tf=True
        )),
        ("clf", LogisticRegression(
            max_iter=2000,
            class_weight="balanced",
            C=c_value,
            n_jobs=-1
        ))
    ])
    return pipeline


# Otimiza o threshold para maximizar F1 no conjunto de validação.
def tune_threshold_for_f1(model, X_val, y_val, thresholds=np.linspace(0.1, 0.9, 81)):

    val_proba = model.predict_proba(X_val)[:, 1]

    best_t = 0.5
    best_f1 = -1.0
    for t in thresholds:
        preds = (val_proba >= t).astype(int)
        f1 = f1_score(y_val, preds)
        if f1 > best_f1:
            best_f1 = f1
            best_t = t

    val_pred_best = (val_proba >= best_t).astype(int)
    return best_t, best_f1, val_proba, val_pred_best


def run_experiment(experiment_name, X_series, y_series, max_features=70000, c_value=3.0, ngram_range=(1,3)):
    """
    Executa:
    - split estratificado
    - treina pipeline
    - otimiza threshold p/ F1
    - imprime classification_report
    - retorna resultados em dict
    """
    X_train, X_val, y_train, y_val = train_test_split(
        X_series, y_series,
        test_size=0.2,
        stratify=y_series,
        random_state=42
    )

    pipeline = make_pipeline(max_features=max_features, c_value=c_value, ngram_range=ngram_range)
    pipeline.fit(X_train, y_train)

    best_t, best_f1, val_proba, val_pred = tune_threshold_for_f1(pipeline, X_val, y_val)

    print("\n" + "="*90)
    print(f"EXPERIMENTO: {experiment_name}")
    print("="*90)
    print(f"max_features={max_features} | C={c_value} | ngram_range={ngram_range} | best_t={best_t:.3f}")
    print(f"F1(val) no melhor threshold: {best_f1:.6f}\n")
    print(classification_report(y_val, val_pred, digits=4))

    # Pequenas estatísticas úteis para relatório
    n_val = len(y_val)
    n_true = int((y_val == 0).sum())
    n_fake = int((y_val == 1).sum())

    return {
        "experiment": experiment_name,
        "max_features": max_features,
        "C": c_value,
        "ngram_range": str(ngram_range),
        "best_threshold": round(float(best_t), 3),
        "f1_val": float(best_f1),
        "val_size": int(n_val),
        "val_true": n_true,
        "val_fake": n_fake
    }


In [4]:
# 1) EXPERIMENTO A (BASE): title+text

results = []

res_base = run_experiment(
    experiment_name="A) BASE: title+text",
    X_series=train["content"],
    y_series=y,
    max_features=70000,
    c_value=3.0,
    ngram_range=(1,3)
)
results.append(res_base)


EXPERIMENTO: A) BASE: title+text
max_features=70000 | C=3.0 | ngram_range=(1, 3) | best_t=0.400
F1(val) no melhor threshold: 0.986977

              precision    recall  f1-score   support

           0     0.9976    0.9941    0.9959      3400
           1     0.9815    0.9925    0.9870      1069

    accuracy                         0.9937      4469
   macro avg     0.9896    0.9933    0.9914      4469
weighted avg     0.9938    0.9937    0.9937      4469



In [5]:
# 2) EXPERIMENTO B (REMOÇÃO): remover 'Reuters'

train["content_no_reuters"] = train["content"].apply(remove_reuters)

res_no_reuters = run_experiment(
    experiment_name="B) REMOÇÃO: title+text (removendo 'Reuters')",
    X_series=train["content_no_reuters"],
    y_series=y,
    max_features=70000,
    c_value=3.0,
    ngram_range=(1,3)
)
results.append(res_no_reuters)


EXPERIMENTO: B) REMOÇÃO: title+text (removendo 'Reuters')
max_features=70000 | C=3.0 | ngram_range=(1, 3) | best_t=0.490
F1(val) no melhor threshold: 0.982654

              precision    recall  f1-score   support

           0     0.9938    0.9953    0.9946      3400
           1     0.9850    0.9804    0.9827      1069

    accuracy                         0.9917      4469
   macro avg     0.9894    0.9878    0.9886      4469
weighted avg     0.9917    0.9917    0.9917      4469



In [6]:
# 3) EXPERIMENTO C (METADADO): incluir 'subject'

train["subject_clean"] = train["subject"].fillna("unknown").astype(str).str.lower().str.strip()

train["content_with_subject"] = (
    "[SUBJ] " + train["subject_clean"] + " " + train["content"]
)

res_subject = run_experiment(
    experiment_name="C) METADADO: title+text + subject (prefix token)",
    X_series=train["content_with_subject"],
    y_series=y,
    max_features=70000,
    c_value=3.0,
    ngram_range=(1,3)
)
results.append(res_subject)


EXPERIMENTO: C) METADADO: title+text + subject (prefix token)
max_features=70000 | C=3.0 | ngram_range=(1, 3) | best_t=0.440
F1(val) no melhor threshold: 0.995800

              precision    recall  f1-score   support

           0     0.9994    0.9979    0.9987      3400
           1     0.9935    0.9981    0.9958      1069

    accuracy                         0.9980      4469
   macro avg     0.9964    0.9980    0.9972      4469
weighted avg     0.9980    0.9980    0.9980      4469



In [7]:
# 4) EXPERIMENTO D: subject + sem Reuters
# Esse experimento ajuda a separar efeitos: subject vs Reuters (proxy editorial)

train["content_with_subject_no_reuters"] = (
    "[SUBJ] " + train["subject_clean"] + " " + train["content_no_reuters"]
)

res_subject_no_reuters = run_experiment(
    experiment_name="D) METADADO+ABLATION: subject + (removendo 'Reuters')",
    X_series=train["content_with_subject_no_reuters"],
    y_series=y,
    max_features=70000,
    c_value=3.0,
    ngram_range=(1,3)
)
results.append(res_subject_no_reuters)


EXPERIMENTO: D) METADADO+ABLATION: subject + (removendo 'Reuters')
max_features=70000 | C=3.0 | ngram_range=(1, 3) | best_t=0.430
F1(val) no melhor threshold: 0.995336

              precision    recall  f1-score   support

           0     0.9994    0.9976    0.9985      3400
           1     0.9926    0.9981    0.9953      1069

    accuracy                         0.9978      4469
   macro avg     0.9960    0.9979    0.9969      4469
weighted avg     0.9978    0.9978    0.9978      4469



In [8]:
# 5) TABELA FINAL DE RESULTADOS
results_df = pd.DataFrame(results).sort_values(by="f1_val", ascending=False)
display(results_df)

,experiment,max_features,C,ngram_range,best_threshold,f1_val,val_size,val_true,val_fake
2,C) METADADO: title+text + subject (prefix token),70000,3.0,"(1, 3)",0.44,0.995800,4469,3400,1069
3,D) METADADO+ABLATION: subject + (removendo 'Re...,70000,3.0,"(1, 3)",0.43,0.995336,4469,3400,1069
0,A) BASE: title+text,70000,3.0,"(1, 3)",0.40,0.986977,4469,3400,1069
1,B) REMOÇÃO: title+text (removendo 'Reuters'),70000,3.0,"(1, 3)",0.49,0.982654,4469,3400,1069


# Conclusão

Os resultados evidenciam a presença de sinais de viés editorial no dataset. A inclusão da variável subject como metadado textual elevou substancialmente o desempenho (F1 = 0.9958), indicando que o modelo passa a explorar informações relacionadas à origem ou estilo editorial das notícias, caracterizando potencial leakage de fonte. De forma complementar, a remoção do termo “Reuters” reduziu o desempenho (F1 = 0.9827), confirmando que marcadores de agência jornalística atuam como fortes indícios discriminativos no corpus.

Apesar dos ganhos observados com metadados, optou-se por **não utilizar a variável subject nos modelos subsequentes**, priorizando uma abordagem mais robusta e generalizável baseada exclusivamente no conteúdo textual. Esse experimento também confirma que a decisão metodológica adotada no baseline, de não utilizar a variável subject, foi adequada, evitando dependência de sinais de origem editorial e reduzindo riscos de sobreajuste.